<a href="https://colab.research.google.com/github/DhaneshTikoo/FacialDataProject/blob/main/FacialDataProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

Intro about what this is and how to use it

#File Locations


In your Google Drive, make a folder called "Facial Data"

Place all the Facial Data files in your Google Drive. Make sure they are in the folder you just

If you want to place the files somewhere else, you can modify the file paths below.

In [76]:
# setting default file paths
myPath = '/content/drive/MyDrive/Facial Data'
outputPath = myPath + '/output'

#Start

Now, press the "Runtime" button and click "Run All"
If Google gives you warnings, click "Run anyways."
Additionally, you will need to connect your Google Drive to Google Colab, so make sure to sign in to Google when prompted to mount your Google Drive.

#Setup

##Imports

In [137]:
# importing needed modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from operator import itemgetter
import os

In [144]:
# if mpld3 is not already installed, install it
installedPackages = !pip list -v
installed = False
for package in installedPackages:
  if package[0:5] == 'mpld3':
    installed = True
if installed == False:
  !pip install mpld3

# import mpld3
import mpld3

##Setup Functions

In [78]:
def importData(myfile, myPath):
  """
  imports the data from the file to a dataframe
  myFile: string
    the name of the text file where the data is located, including the filetype suffix
  myPath: string
    the path where the file is stored

  returns:
    imported dataframe
  """
  pathFull = myPath + "/" + myfile
  data = pd.read_csv(pathFull, header = None, skiprows=22)
  data.columns = my_cols
  return data

In [79]:
def derivative(x,y):
  """
  inputs x and y lists and returns the x and y lists for that graph's derivative:
  reduces the length of x by 1 and takes the derivative of y
  x: list, array
  y: list, array
    the x and y lists for a graph

  returns:
    the modified x and y lists
  """
  dydx = np.diff(y)/np.diff(x)
  dydx_x = x[:-1]
  return dydx_x, dydx

In [80]:
def generateDerivativeFrames(df):
  """
  creates dataframes for the derivatives of a dataframe's columns
  data: dataframe
    the name of the text file where the data is located, including the filetype suffix

  returns:
    der1: the first derivative of the columns of data
    der2: the second derivative of the columns of data
  """
  der1 = pd.DataFrame()
  der2 = pd.DataFrame()

  # create derivative 1
  der1 = pd.concat([der1, df["bs"][:-1]], axis=1)
  der1 = pd.concat([der1, df[Milliseconds][:-1]], axis=1)
  for col in df.columns[2:]:
    column = df[col]
    x = []
    x, column_der = derivative(df[Milliseconds], column)
    column_der = pd.Series(column_der)
    der1 = pd.concat([der1, column_der.rename(col)], axis=1)

  # create derivative 2
  der2 = pd.concat([der2, df["bs"][:-2]], axis=1)
  der2 = pd.concat([der2, df[Milliseconds][:-2]], axis=1)
  for col in der1.columns[2:]:
    column = der1[col]
    x = []
    x, column_der2 = derivative(der1[Milliseconds], column)
    column_der2 = pd.Series(column_der2)
    der2 = pd.concat([der2, column_der2.rename(col)], axis=1)

  return der1, der2

##Mount Drive & Make Output Directory

In [81]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

# make output dir
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Establish Variables

In [82]:
# creating a default dataframe
data = pd.DataFrame()

In [83]:
# setting column titles to correspond to the blendshapes they represent
my_cols = ['bs', 'Milliseconds', 'head x', 'head y', 'head z', 'EA x', 'EA y',
       'EA z', 'LE x', 'LE y', 'RE x', 'RE y', 'browInnerUp', 'browDown_L',
       'browDown_R', 'browOuterUp_L', 'browOuterUp_R', 'eyeLookUp_L',
       'eyeLookUp_R', 'eyeLookDown_L', 'eyeLookDown_R', 'eyeLookIn_L',
       'eyeLookIn_R', 'eyeLookOut_L', 'eyeLookOut_R', 'eyeBlink_L',
       'eyeBlink_R', 'eyeSquint_L', 'eyeSquint_R', 'eyeWide_L', 'eyeWide_R',
       'cheekPuff', 'cheekSquint_L', 'cheekSquint_R', 'noseSneer_L',
       'noseSneer_R', 'jawOpen', 'jawForward', 'jawLeft', 'jawRight',
       'mouthFunnel', 'mouthPucker', 'mouthLeft', 'mouthRight',
       'mouthRollUpper', 'mouthRollLower', 'mouthShrugUpper',
       'mouthShrugLower', 'mouthClose', 'mouthSmile_L', 'mouthSmile_R',
       'mouthFrown_L', 'mouthFrown_R', 'mouthDimple_L', 'mouthDimple_R',
       'mouthUpperUp_L', 'mouthUpperUp_R', 'mouthLowerDown_L',
       'mouthLowerDown_R', 'mouthPress_L', 'mouthPress_R', 'mouthStretch_L',
       'mouthStretch_R', 'tongueOut']

In [84]:
# defining pairs of columns
colPairs = {
  'EulerAngles': ['EA x', 'EA y'],
  'EyeX':['LE x', 'RE x'],
  'EyeY': ['LE y', 'RE y'],
  'browDown':['browDown_L', 'browDown_R'],
  'browOuterUp':['browOuterUp_L', 'browOuterUp_R'],
  'eyeLookUp':['eyeLookUp_L', 'eyeLookUp_R'],
  'eyeLookDown':['eyeLookDown_L', 'eyeLookDown_R'],
  'eyeLookIn':['eyeLookIn_L', 'eyeLookIn_R'],
  'eyeLookOut':['eyeLookOut_L', 'eyeLookOut_R'],
  'eyeBlink':['eyeBlink_L', 'eyeBlink_R'], 
  'eyeSquint':['eyeSquint_L', 'eyeSquint_R'], 
  'eyeWide':['eyeWide_L', 'eyeWide_R'],
  'cheekSquint':['cheekSquint_L', 'cheekSquint_R'], 
  'noseSneer':['noseSneer_L', 'noseSneer_R'], 
  'jaw':['jawLeft', 'jawRight'],
  'mouth':['mouthLeft', 'mouthRight'],
  'mouthRoll':['mouthRollUpper', 'mouthRollLower'] ,
  'mouthShrug':['mouthShrugUpper',  'mouthShrugLower'], 
  'mouthSmile':['mouthSmile_L', 'mouthSmile_R'],
  'mouthFrown':['mouthFrown_L', 'mouthFrown_R'], 
  'mouthDimple':['mouthDimple_L', 'mouthDimple_R'],
  'mouthUpper':['mouthUpperUp_L', 'mouthUpperUp_R'],
  'mouthLower':['mouthLowerDown_L','mouthLowerDown_R'], 
  'mouthPress':['mouthPress_L', 'mouthPress_R'], 
  'mouthStretch':['mouthStretch_L','mouthStretch_R']
}

In [85]:
# defines Milliseconds as the string of the millisecond column
Milliseconds = my_cols[1]

##Import Data

In [86]:
#importing the data tables to dataframes
dfs = []
dfsNames = []
dfs_vel = []
dfs_accel = []
i=0

for filename in os.listdir(myPath):
    if filename.endswith("txt"): 
        dfs.append(importData(filename, myPath))
        dfsNames.append(filename[:-4])
        tempData1, tempData2 = generateDerivativeFrames(dfs[i])
        dfs_vel.append(tempData1)
        dfs_vel.append(tempData2)
        i+=1

#Graphing & Analysis

##Graph Column or ColPair Section (FINAL)

In [87]:
def GraphColumnsSection(colTitles = [], df = data, xstart = 0, xend = None, together = False):
  """
  graphs the given columns from the given dataframe, with a restricted x axis
  colTitles: list, array
    a list or array of all names of columns which will be graphed
  df: dataframe
    the dataframe that the columns will be from
  together: boolean
    Whether or not the graphs will be overlayed or not
  xstart: integer
    The lowest value for the x axis
  xend: integer
    The highest value for the x axis
  """
  # if xend is not input
  if xend == None:
    # set the maximum x value to the highest it can be with the given dataframe
    xend = max(df[Milliseconds])
  # if together is True, graph both graphs on the same axis
  if together:
    fig, ax = plt.subplots(1, 1, figsize = (8, 6))
    for colTitle in colTitles:
      ax.plot(df[Milliseconds], df[colTitle], label = colTitle)
      ax.legend()
      ax.set_xlim(xstart, xend)
  # otherwise, graph them on different axes
  else:
    fig, ax = plt.subplots(len(colTitles), 1, figsize = (8, 6*len(colTitles)))
    for i in range(len(colTitles)):
      ax[i].plot(df[Milliseconds], df[colTitles[i]], label = colTitles[i])
      ax[i].legend()
      ax[i].set_xlim(xstart, xend)

In [88]:
def GraphColPairsSection(pairs = [], df = data, xstart = 0, xend = None):
  """
  graphs the given column pairs from the given dataframe, with a restricted x axis
  pairs: list, array
    a list or array of all names of column pairs which will be graphed
  df: dataframe
    the dataframe that the columns will be from
  xstart: integer
    The lowest value for the x axis
  xend: integer
    The highest value for the x axis
  """
  # if xend is not input
  if xend == None:
    # set the maximum x value to the highest it can be with the given dataframe
    xend = max(df[Milliseconds])
  # create figure and axis
  fig, ax = plt.subplots(1, 1, figsize = (8, 6))
  # graph each pair with the restricted x axis
  for pairName in pairs:
    colPairs[pairName][0]
    ax.plot(df[Milliseconds], df[colPairs[pairName][0]], label = colPairs[pairName][0])
    ax.plot(df[Milliseconds], df[colPairs[pairName][1]], label = colPairs[pairName][1])
    ax.legend()
    ax.set_xlim(xstart, xend)

##Graph All (FINAL)

In [89]:
def graphAll(dfs = [], titles = [], skinnyGraph = True, scaled = False):
  """
  graphs all of the columns for each of the given dataframes
  the format of the graph depends on the 'skinnyGraph' variable
  dfs: list
    the list of dataframes which will have thier columns graphed
  titles: list
    the list of titles corresponding to the dataframes
    the first element of titles will be used to label the first element of dfs, etc.
  skinnyGraph: boolean
    if True, each dataframe will occupy only one column
    if False, the graphs will still be next to each other, but will occupy multiple columns
  scaled: boolean
    if True, the y axis will go from 0 to 1

  returns:
    the graph of all the columns of each dataframe
  """
  # calculates graph size based on the amount of dataframes input
  if skinnyGraph == True:
    totalRows = len(dfs[0].columns)-2
    totalColumns = len(dfs)
    figsize=(8*totalColumns, 6*totalRows)
  else:
    totalGraphsNum = len(dfs)*(len(dfs[0].columns)-2)
    totalRows = math.floor(math.sqrt(totalGraphsNum))
    totalColumns = math.ceil(totalGraphsNum/totalRows)
    while totalColumns % len(dfs) != 0:
      totalColumns+=1
    figsize=(8*totalColumns, 6*totalRows)

  # create the figure and axis
  fig, ax = plt.subplots(totalRows, totalColumns, figsize = figsize)
  # if scaled is true, limit the y axis
  if scaled == True:
    plt.setp(ax, ylim = (0, 1))
  # create the graph and save it to a file
  row = 0
  column = 0
  colTitles = dfs[0].columns[2:]
  # if there is more than one column, create the graph via this logic
  if totalColumns > 1:
    for col in colTitles:
      for i in range(0, len(dfs)):
        x=dfs[i][Milliseconds]
        y=dfs[i][col]

        ax[row, column+i].plot(x,y, label=col)
        ax[row, column+i].set_title(f'{titles[i]}')
        ax[row, column+i].legend()
      row+=1
      if row%totalRows==0:
        row = 0
        column+=len(dfs)
  # otherwise, use this logic
  else:
    for col in colTitles:
      x=dfs[0][Milliseconds]
      y=dfs[0][col]

      ax[row].plot(x,y, label=col)
      ax[row].set_title(f'{titles[0]}')
      ax[row].legend()
      row+=1

  return fig

In [90]:
def graphAllToFile(dfs = [], titles = [], skinnyGraph = True, scaled = False):
  """
  graphs all of the columns for each of the given dataframes and outputs them to a file
  the format of the graph depends on the 'skinnyGraph' variable
  dfs: list
    the list of dataframes which will have thier columns graphed
  titles: list
    the list of titles corresponding to the dataframes
    the first element of titles will be used to label the first element of dfs, etc.
  skinnyGraph: boolean
    if True, each dataframe will occupy only one column
    if False, the graphs will still be next to each other, but will occupy multiple columns
  scaled: boolean
    if True, the y axis will go from 0 to 1
  """
  fig = graphAll(dfs,names)
  nametotal = ""
  for index in range(len(names)):
    nametotal = nametotal + names[index] + "-"
  saveGraph(fig, f"{nametotal}All Graphs")

##Fourier Transform Functions (FINAL)

In [91]:
def compareFourierTransforms(dfs = [], titles = [], skinnyGraph = True):
  """
  outputs graphs of the fourier transforms of each column in the input dataframes
  dfs: list
    the list of dataframes that will have their columns' fourier transforms compared
  titles: list
    the names of the corresponding output columns
    the 1st name will be attached to the first dataframe, and so on
  skinnyGraph: boolean
    If true, the graph will have as many columns as dataframes input
    If false, the graph will instead be more square-shaped
  returns:
    a graph with the fourier transforms for each column in the given dataframes
  """
  # seting graph sizes
  if skinnyGraph == True:
    totalRows = len(my_cols[12:])
    totalColumns = len(dfs)
    figsize=(8*totalColumns, 6*totalRows)
  else:
    totalGraphsNum = len(dfs)*len(my_cols[12:])
    totalRows = math.floor(math.sqrt(totalGraphsNum))
    totalColumns = math.ceil(totalGraphsNum/totalRows)
    while totalColumns % len(dfs) != 0:
       totalColumns+=1
    figsize=(8*totalColumns, 6*totalRows)

  fig, ax = plt.subplots(totalRows, totalColumns, figsize = figsize)

  # creating the fourier transforms at their appropriate location in the graph
  for i in range(len(dfs)):
    for j in range(len(my_cols[12:])):
      FFT = np.fft.fft(dfs[i][my_cols[j+12]])
      freq = np.arange(len(dfs[i][Milliseconds]))/dfs[i][Milliseconds][len(dfs[i][Milliseconds])-1] # scale by dividing by max time
      ax[j][i].plot(freq,np.abs(FFT))
      ax[j][i].set_title(f"{titles[i]}, {my_cols[j+12]}")

  return fig

In [92]:
def fourierTransform(df = data):
  """
  displays all graphs for the fourier transforms in a given dataframe
  data: dataframe
    the dataframe that has columns that will be input into the fourier transform
  returns:
    multiple graphs, each of a column that has been input into the fourier transform
  """
  for col in my_cols[2:]:
    SecondsfromNano = df[Milliseconds]/1000
    
    FFT = np.fft.fft(df[col])
    freq = np.arange(len(SecondsfromNano))/SecondsfromNano[len(SecondsfromNano)-1] # scale by dividing by max time
    fig, ax = plt.subplots(1,1, figsize = (10,8))
    ax.set_xlim(0, 30)
    ax.plot(freq,np.abs(FFT))
    ax.set_title(col)

In [93]:
def fourierTransformInterval(df = data, col = "LE x", startinit = 0, endinit = 10000, xlim = (0, 30), ylim = None):
  """
  performs a fourier transform on a certain interval for a specific column in a specific dataframe
  df: dataframe
    the dataframe input
  col: string
    the name of the column to be input
  startinit: integer
    the lower bound for the fourier transform
  endinit: integer
    the higher bound for the fourier transform
  xlim: tuple
    the bounds for the x-axis
  ylim: tuple
    the bounds for the y-axis
    If not input, the y axis will be automatically assigned
  """
  start = findInDf(startinit, df)
  end = findInDf(endinit, df)

  SecondsfromNano = df[Milliseconds][start:end]/1000

  FFT = np.fft.fft(df[col][start:end])
  freq = np.arange(len(SecondsfromNano))/(SecondsfromNano[end-1]-SecondsfromNano[start]) # scale by dividing by max time
  fig, ax = plt.subplots(1,1, figsize = (10,8))
  ax.set_xlim(xlim)
  if ylim != None:
    ax.set_ylim(ylim)
  ax.plot(freq,np.abs(FFT))
  ax.set_title(col)
  plt.show()

#Averages

##Internal Functions

In [94]:
def covertValuesToArray(dictionary = colPairs):
  propertyColumn = list(dictionary.values())
  expandedColumn = []
  for i in propertyColumn:
    for j in range(2):
      expandedColumn.append(i[j])
  return expandedColumn

In [95]:
def averageData(dataTitle=[Milliseconds]):
  """
  prints the average value of all the entries of a column for each of the given colums in data
  dataTitle: list
    a list of column names

  returns:
    none
    
  example:
    averageData([Milliseconds, 'browDown_L'])
  """
  for cols in dataTitle:
    #if bs
    if cols == my_cols[0]:
      cols = Milliseconds
      print('Error: Cannot average \'bs\'')
    print(f"{cols}: {np.average(data[cols])}")

In [96]:
def findInDf(number=0, df = data):
  while True:
    try:
      start = list(df[Milliseconds]).index(number)
      break
    except:
      number+=1

  return start

In [97]:
def averageValuesFrame(dfs = [], names = [],individualColPairs = False):
  if individualColPairs == False:
    expandedColumn = covertValuesToArray(colPairs)
    avgValueFrame = pd.DataFrame(expandedColumn, columns = ["Blendshapes"])

    for i in range(len(dfs)):
      avgValueList = []
      for j in expandedColumn:
        avgColValue = np.average(dfs[i][j])
        avgValueList.append(avgColValue)
      avgValueList = pd.Series(avgValueList)
      avgValueFrame = pd.concat([avgValueFrame, avgValueList.rename(f"{names[i]}")], axis=1)
    return avgValueFrame

  if individualColPairs == True:
    propertyColumn = list(colPairs.keys())
    avgValueFrame = pd.DataFrame(propertyColumn, columns = ["Blendshape Pairs"])

    for i in range(len(dfs)):
      avgValueList = []
      for j in propertyColumn:
        colName1 = colPairs[j][0]
        colName2 = colPairs[j][1]

        avgColValue1 = np.average(dfs[i][colName1])
        avgColValue2 = np.average(dfs[i][colName2])
        avgColValue = np.average([avgColValue1, avgColValue2])

        avgValueList.append(avgColValue)

      avgValueList = pd.Series(avgValueList)
      avgValueFrame = pd.concat([avgValueFrame, avgValueList.rename(f"{names[i]}")], axis=1)
    return avgValueFrame

##Average over an Interval

In [98]:
def averageDataInterval(dataTitle=[Milliseconds], start=0, end=3000):
  """
  prints the average of the values for each of the given columns over the given interval
  dataTitle: list
    a list of strings that are column names
  start: int
    the starting number in Milliseconds to begin averaging points at
  end: int
    the ending number in Milliseconds to begin averaging points at
  
  returns:
    none
    
  example:
    averageDataInterval([Milliseconds, 'browDown_L'], 0, 2000)
  """
  # out of bounds error checking
  if start>3008 or start<0 or end>3008 or start>=end:
    print('Error: Out of Bounds')
    return None

  # correct starting values in Milliseconds
  start = findInDf(start, data)
  end = findInDf(end, data)
  # print interval range
  print(f"Averages over the interval [{data[Milliseconds][start]}, {data[Milliseconds][end]}]:")
  # for each column calculate and print averages
  for cols in dataTitle:
    #bs column error checking
    if cols == my_cols[0]:
      cols = Milliseconds
      print('Error: Cannot average \'bs\'')
    # calculate and print average value for a given column
    averageOverInterval = np.average(data[cols][start:end])
    print(f"{cols}: {averageOverInterval}")

##Average To File (FINAL)

In [99]:
def averageValuesToFile(dfs = [], names = [], individualColPairs = False):
  """
  outputs the average values of the given dataframes to a file
  """
  avgValuesFrame = averageValuesFrame(dfs, names, individualColPairs)
  nametotal = ""
  for index in range(len(names)):
    nametotal = nametotal + names[index] + "-"
  avgValuesFrame.to_csv(f"{outputPath}/{nametotal}Average Values.csv")

#Symmetry Analysis

##Internal Functions

In [100]:
zFillValue = 5

In [101]:
def peaksAndValleys(dataTitle, ax):
  """
  creates a list of all points in data where each point is equal to the points adjacent to it
  dataTitle: string
    the name of a column in data
  ax:
    the axis of a given graph
    multiple axis elements are stored as an np.ndarray

  returns:
    a list of points, where each point is equal to the points adjacent to it
  """
  pav = []
  # if ax is a list, set it to its first graph
  if type(ax) is np.ndarray:
    ax = ax[0]
  # for each point in the given column
  for n in range(1, len(data[dataTitle])-1):
    # if this point is increaseing, color it red
    if data[dataTitle][n-1] < data[dataTitle][n] and data[dataTitle][n] < data[dataTitle][n+1]:
      ax.scatter(data[Milliseconds][n], data[dataTitle][n], color = 'Red')
      pass
    # if this point is decreasing, color it green
    elif data[dataTitle][n-1] > data[dataTitle][n] and data[dataTitle][n] > data[dataTitle][n+1]:
      ax.scatter(data[Milliseconds][n], data[dataTitle][n], color = 'Green')
      pass
    # if this graph is neither increasing nor decreasing, append it to pav
    else:
      pav.append((data[Milliseconds][n], data[dataTitle][n]))
  # color all points in pav black
  for n in pav:
    ax.scatter(n[0], n[1], color = 'Black')
  return pav

In [102]:
def peaksAndValleysDF(dataTitle, df, ax = None, displaygraph = False):
  """
  creates a list of all points in data where each point is equal to the points adjacent to it
  dataTitle: string
    the name of a column in data
  ax:
    the axis of a given graph
    multiple axis elements are stored as an np.ndarray

  returns:
    a list of points, where each point is equal to the points adjacent to it
  """
  pav = []
  # if ax is a list, set it to its first graph
  if type(ax) is np.ndarray:
    ax = ax[0]
  # for each point in the given column
  for n in range(1, len(df[dataTitle])-1):
    # if this point is increaseing, color it red
    if df[dataTitle][n-1] < df[dataTitle][n] and df[dataTitle][n] < df[dataTitle][n+1]:
      if displaygraph == True:
        ax.scatter(df[Milliseconds][n], df[dataTitle][n], color = 'Red')
      else:
        pass
    # if this point is decreasing, color it green
    elif df[dataTitle][n-1] > df[dataTitle][n] and df[dataTitle][n] > df[dataTitle][n+1]:
      if displaygraph == True:
        ax.scatter(df[Milliseconds][n], df[dataTitle][n], color = 'Green')
      else:
        pass
    # if this graph is neither increasing nor decreasing, append it to pav
    else:
      pav.append((df[Milliseconds][n], df[dataTitle][n]))
  # color all points in pav black
  if displaygraph == True:
    # add the graph line to the graph
    ax.plot(df[Milliseconds], df[dataTitle], zorder = 0, label = dataTitle)
    for n in pav:
      ax.scatter(n[0], n[1], color = 'Black')
    ax.legend()
    return ax
  if displaygraph == False:
    return pav

In [103]:
def createPavFrame(dataTitle, pav):
  """
  creates a DataFrame that has all of the peaks and valleys in pav, one peak or valley for each column
  dataTitle: string
    the name of a column in data
  pav:
    the output of the pav function for the given dataTitle

  returns:
    a DataFrame that has all of the peaks and valleys in pav, one peak or valley for each column
  """
  pavFrame = pd.DataFrame()

  skipValue = (max(data[dataTitle])-min(data[dataTitle]))/8
  i = 0
  tpoint = 0
  for n in range(len(pav)-1):
    index = findInDf(pav[n][0], data)
    index2 = findInDf(pav[n+1][0], data)
    if abs(data[dataTitle][index] - data[dataTitle][index2]) > skipValue or n==len(pav)-3:
      j = n+1
      ser = pd.Series(pav[i:j], name=f"Peak/Valley {str(tpoint).zfill(zFillValue)}")
      tpoint=tpoint+1
      pavFrame = pd.concat([pavFrame, ser], axis=1)
      i = j
  
  return pavFrame

In [104]:
def tupleFixer(column):
  """
  creates an array with one-element entries
  dataTitle: string
    the name of a column in data
  ax:
    the axis of a given graph
    multiple axis elements are stored as an np.ndarray

  returns:
    an array that is the same as the input column, but replaces all tuples with only their x-value
  """
  new = []
  for x in column:
    if x is np.nan:
      new.append(x)
    else:
      new.append(x[0])
  new = np.array(new)
  return new

In [105]:
def pavMidpoint(column):
  """
  finds the x-value of the midpoint of a given column
  dataTitle: string
    the name of a column in data

  returns:
    the x-value of the midpoint of a given column
  """
  new = tupleFixer(column)

  midPoint = np.nanmedian(new)
  #if even number of elements, takes the lower of the two middle values
  midPoint = new[new<=midPoint].max()

  return midPoint

In [106]:
def symmetryValue(column, dataTitle, checkRange=10):
  """
  assigns a value to quantify the symmetry of a given dataTitle
  column:
    a column in a given pavFrame
  dataTitle: string
    the name of a column in data
  checkRange:
    the amount of points to the left and right that the function should search

  returns:
    returns a value to quantify the symmetry of a given dataTitle
  """
  midPointX = pavMidpoint(column)
  midPointIndex = findInDf(midPointX, data)
  midPointY = data[dataTitle][midPointIndex]

  symmetryRange = data[dataTitle][midPointIndex-checkRange:midPointIndex+checkRange+1]

  rightSideInc = None
  leftSideInc = None
  n = 1

  while rightSideInc == None or leftSideInc == None and n<=checkRange+1:
    if symmetryRange[midPointIndex] < symmetryRange[midPointIndex+n]:
      rightSideInc = "Increasing"
    elif symmetryRange[midPointIndex] > symmetryRange[midPointIndex+n]:
      rightSideInc = "Decreasing"
    elif symmetryRange[midPointIndex] == symmetryRange[midPointIndex+n]:
      rightSideInc = None
    if symmetryRange[midPointIndex] < symmetryRange[midPointIndex-n]:
      leftSideInc = "Increasing"
    elif symmetryRange[midPointIndex] > symmetryRange[midPointIndex-n]:
      leftSideInc = "Decreasing"
    elif symmetryRange[midPointIndex] == symmetryRange[midPointIndex-n]:
      leftSideInc = None
    n+=1

  for n in range(2, checkRange):
    if symmetryRange[midPointIndex+n-1] < symmetryRange[midPointIndex+n]:
      currentChange = "Increasing"
    elif symmetryRange[midPointIndex+n-1] > symmetryRange[midPointIndex+n]:
      currentChange = "Decreasing"
    elif symmetryRange[midPointIndex+n-1] == symmetryRange[midPointIndex+n]:
      currentChange = rightSideInc
    if currentChange != rightSideInc:
      checkRange = n-1
      break
    if symmetryRange[midPointIndex-n+1] < symmetryRange[midPointIndex-n]:
      currentChange = "Increasing"
    elif symmetryRange[midPointIndex-n+1] > symmetryRange[midPointIndex-n]:
      currentChange = "Decreasing"
    elif symmetryRange[midPointIndex-n+1] == symmetryRange[midPointIndex-n]:
      currentChange = leftSideInc
    if currentChange != leftSideInc and n-1 <= checkRange:
      checkRange = n-1
      break

  symmetryRangeNew = data[dataTitle][midPointIndex-checkRange:midPointIndex+checkRange+1]

  symmetryNumbers = []
  for n in range(1,checkRange+1):
    x = abs(symmetryRangeNew[midPointIndex]-symmetryRangeNew[midPointIndex-n])
    symmetryNumbers.append(x)
    x = abs(symmetryRangeNew[midPointIndex+n]-symmetryRangeNew[midPointIndex])
    symmetryNumbers.append(x)
  return np.mean(symmetryNumbers)

In [107]:
def symmetryValuesPosition(dataTitle, df = data):
  tempSymmetryValues = []
  for i, j in zip(df[dataTitle[0]], df[dataTitle[1]]):
    if i+j==0:
      tempSymmetryValues.append(0)
    else:
      tempSymmetryValues.append(np.round(abs((j-i)/((i+j)/2)), 4))
  return np.mean(tempSymmetryValues)

In [108]:
def allSymmetry(dfs = [data]):
  for i in dfs:
    allSymmArr = []
    for key, value in colPairs.items():
      allSymmArr.append(np.round(100*compareSymmetryPosition(value, i), 2))
    for n, key in zip(range(len(colPairs.keys())),colPairs.keys()):
      compareSymmetryPosition(colPairs[key], i)
      print(f"{key}: {allSymmArr[n]}")
    print(f"Mean average value: {np.round(np.mean(allSymmArr), 2)}")
    print()

##AnalyzeSymmetry Function
A function that shows the peaks and valleys of a column

In [109]:
def analyzeSymmetry(dataTitle='jawForward'):
  """
  creates and displays a dictionary for the peaks and valleys of data, where
  the key is the peak number in the format 'Peak/Valley XXX', and the value is
  a list with the x-value of where the peak or valley appears and the 
  symmetry value for that peak or valley
  dataTitle:
    the column in data which is analyzed

  returns:
    none, displays dictionary of peaks and valleys and a graph that shows
    which points are increasing, which remain the same, and which ones are decreasing
  """

  #graphing
  fig, ax = graphPlotData([dataTitle], False, False)

  #creating pav AND adding the graphical dots
  pav = peaksAndValleys(dataTitle, ax)

  #creating & populating pavFrame
  pavFrame = createPavFrame(dataTitle, pav)

  #creating pavdict
  pavDict = {}
  # for each peak or valley
  for n in range(len(pavFrame.columns)):
    # create a string that is 3 characters long, consisting of left zeros in empty places
    # this string will be the title of the corresponding peak or valley midpoint
    sortedN = str(n).zfill(zFillValue)
    #establish title
    titleOfPoint=f"Peak/Valley {sortedN}"
    # find the midpoint of the entries in the given column of pavFrame, converted to int
    midpointOfPavFrameColumn = int(pavMidpoint(pavFrame[titleOfPoint]))
    # finding the symmetry value that corresponds to dataTitle in data
    # then, round the symmetry value to 4 decimal places
    dataTitleSymmetryValue = symmetryValue(pavFrame[titleOfPoint], dataTitle)
    dataTitleSymmetryValueRounded = np.round(dataTitleSymmetryValue, 4)
    # create an entry in pavDict with a title "titleofPoint" that corresponds to (x, symmetry)
    # where x is the time value for the midpoint of given peak/valley
    # where symmetry is the symmetry value for the given peak/valley
    pavDict[titleOfPoint] = [midpointOfPavFrameColumn, dataTitleSymmetryValueRounded]
  
  display(pavDict)
# ---------------------------------------------------------------------------------
# analyzeSymmetry('jawForward')

#Symmetry Comparison

##Compare Symmetry Graph Functions
Functions that display graphs of columns of the data sets for comparison

In [110]:
def compareSymmetryGraphs(dfs = [], titles = [], skinnyGraph = True, scaled = False):
  if skinnyGraph == True:
    totalRows, totalColumns, figsize = skinnyGraphSizes(dfs)
  else:
    totalRows, totalColumns, figsize = squareGraphSizes(dfs)

  fig, ax = plt.subplots(totalRows, totalColumns, figsize = figsize)
  
  if scaled == True:
    plt.setp(ax, ylim = (0, 1))

  graphColPair(dfs, titles, colPairs.keys(), ax, totalRows)
  display(fig)
  return fig

## Symmetry Values Functions
Functions that the symmetry values for comparison between two parts of a colPair

In [111]:
def symmetryValues(dfs = [], names = []):
  propertyColumn = list(colPairs.keys())
  symmValueFrame = pd.DataFrame(propertyColumn, columns = ["Blendshapes"])

  for i in range(len(dfs)):
    symmValueList = []
    for key in colPairs.keys():
      col1 = dfs[i][colPairs[key][0]]
      col2 = dfs[i][colPairs[key][1]]
      symmValue = getSymmetryValue(col1, col2)
      symmValueList.append(symmValue)
    symmValueList = pd.Series(symmValueList)
    symmValueFrame = pd.concat([symmValueFrame, symmValueList.rename(f"{names[i]}")], axis=1)
  return symmValueFrame

##ToFile Functions (FINAL)
Functions that create an output file for graphs

In [112]:
def compareSymmetryGraphsToFile(dfs = [], titles = [], skinnyGraph = True, scaled = False):
  fig = compareSymmetryGraphs(dfs, titles, skinnyGraph, scaled)
  nametotal = ""
  for index in range(len(names)):
    nametotal = nametotal + names[index] + "-"
  saveGraph(fig, f"{nametotal}Symmetry Graph")

In [113]:
def symmetryValuesToFile(dfs = [], names = []):
  symmValueFrame = symmetryValues(dfs, names)
  nametotal = ""
  for index in range(len(names)):
    nametotal = nametotal + names[index] + "-"
  symmValueFrame.to_csv(f"{outputPath}/{nametotal}Symmetry Values.csv")

##Internal Functions

In [114]:
def getSymmetryValue(y1, y2):
  tempSymmetryValues = []
  for i, j in zip(y1, y2):
    if i+j==0:
      tempSymmetryValues.append(0)
    else:
      difference = abs((j-i))
      tempSymmetryValues.append(difference)
  averageSymmetryValue = np.mean(tempSymmetryValues)
  return np.round(averageSymmetryValue, 4)

In [115]:
def skinnyGraphSizes(dfs = []): 
  totalRows = len(colPairs.keys())
  totalColumns = len(dfs)
  figsize=(8*totalColumns, 6*totalRows)
  return totalRows, totalColumns, figsize

In [116]:
def squareGraphSizes(dfs = []):
  totalGraphsNum = len(dfs)*len(colPairs.keys())
  totalRows = math.floor(math.sqrt(totalGraphsNum))
  totalColumns = math.ceil(totalGraphsNum/totalRows)
  while totalColumns % len(dfs) != 0:
      totalColumns+=1
  figsize=(8*totalColumns, 6*totalRows)
  return totalRows, totalColumns, figsize

In [117]:
def saveGraph(fig, name):
  fileName = f'{outputPath}/{name}.png'
  fig.savefig(fileName)

In [118]:
def graphColPair(dfs = [], titles = [],keys = [], ax = "Placeholder", totalRows = 5):
  row = 0
  column = 0

  for key in keys:
    for i in range(len(dfs)):
      colTitles = colPairs[key]
      x1=dfs[i][Milliseconds]
      y11=dfs[i][colTitles[0]]
      y12=dfs[i][colTitles[1]]

      ax[row, column+i].plot(x1,y11, label=colTitles[0])
      ax[row, column+i].plot(x1,y12, label=colTitles[1])
      ax[row, column+i].set_title(f'{titles[i]}')
      ax[row, column+i].legend()
    row+=1
    if row%totalRows==0:
      row = 0
      column+=len(dfs)

#Eye Blink Counting

##Count Blinks & Blinks per Second (FINAL)

In [119]:
def CountBlinks(df = data, tolerance = 0.1, middlePointMax = 10, name = ""):
  """
  counts the amount of blinks in the given dataframe
  does this my counting the amount of times that there is a low point followed by a high point
  a low point is any point that is lower than "tolerance" of the midpoint
  a high point is any point that is higher than "tolerance" of the midpoint
  ex. if the average y value is 0.4:
      any point with a y value lower than 0.2 would be a low point
      any point with a y value higher than 0.6 would be a high point
  if there are more than "midpoint" points within "tolerance" of the middle
  the dataset must be manually reviewed, as the blink number would be innacurate

  returns:
  The amount of blinks and blinks per second for the given dataframe
  """
  middlePoints = 0
  pav_L = peaksAndValleysDF("eyeBlink_L", df)
  pav_R = peaksAndValleysDF("eyeBlink_R", df)
  pav = [pav_L, pav_R]

  if name != "":
    name+=": "

  for i in range(len(pav)):
    highestPav = max(pav[i], key=itemgetter(1))[1]
    lowestPav = min(pav[i], key=itemgetter(1))[1]

    mediumvalue = (highestPav+lowestPav)/2
    lowbound = mediumvalue-tolerance
    highbound = mediumvalue+tolerance

    for j in pav[i]:
      if j[1] > lowbound and j[1] < highbound:
        middlePoints+=1
    if middlePoints>=middlePointMax:
      if i == 0:
        LorR = "L"
      if i == 1:
        LorR = "R"
      print(f"{name}Please review eyeBlink_{LorR} Manually")
    else:
      blinks = 0
      seconds = max(df[Milliseconds])/1000

      point0 = 0
      point1 = 1
      for y in pav[i][:-1]:
        y1 = pav[i][point0][1]
        y2 = pav[i][point1][1]
        if y1 < lowbound and y2 > highbound:
          blinks+=1
        point0+=1
        point1+=1

      bps = blinks/seconds
      if i == 0:
        LorR = "L"
      if i == 1:
        LorR = "R"
      print(f"{name}eyeBlink_{LorR} has {blinks} blinks, for a total of {bps} blinks per second.")
  # return mediumvalue

In [120]:
def showBlinkGraphs(df = data):
  """
  Graphs the eyeBlink blendshapes for the given dataframe.
  If the point has a positive slope, it is red
  If the point has a negative slope, it is green
  Otherwise, it is black
  df: dataframe
    the dataframe of which to graph the blink blendshapes
  """
  fig, ax = plt.subplots(2, 1, figsize = (8, 12))
  peaksAndValleysGraph("eyeBlink_L", df, fig, ax[0])
  peaksAndValleysGraph("eyeBlink_R", df, fig, ax[1])

##Blink Graphs To File (FINAL)


In [121]:
def BlinkGraphsToFile(dfs = [], names = []):
  """
  Graphs the eyeBlink blendshapes for the given dataframes and returns them in a file
  If the point has a positive slope, it is red
  If the point has a negative slope, it is green
  Otherwise, it is black
  df: dataframe
    the dataframe of which to graph the blink blendshapes
  """
  for i in range(len(dfs)):
    fig, ax = plt.subplots(2, 1, figsize = (8, 12))
    peaksAndValleysGraph("eyeBlink_L", dfs[i], fig, ax[0])
    ax[0].set_title("eyeBlink_L")
    peaksAndValleysGraph("eyeBlink_R", dfs[i], fig, ax[1])
    ax[1].set_title("eyeBlink_R")
    saveGraph(fig, f"{names[i]} Blink Graphs")

##Internal Functions

In [122]:
def peaksAndValleysGraph(dataTitle, df, fig = None, ax = None):
  """
  visualizes on a graph which points in the given column of df are the decreasing, incraesing, or neither
  dataTitle: string
    the column to graph
  df: dataframe
    the dataframe from which to graph the column
  ax: axis
    the axis to graph on
  """
  if fig == None or ax == None:
    fig, ax = plt.subplots(1, 1, figsize = (8, 6))
  ax.plot(df[Milliseconds],df[dataTitle],zorder = -1)
  # for each point in the given column
  for n in range(1, len(df[dataTitle])-1):
    # if this point is increasing, color it red
    if df[dataTitle][n-1] < df[dataTitle][n] and df[dataTitle][n] < df[dataTitle][n+1]:
      ax.scatter(df[Milliseconds][n], df[dataTitle][n], color = 'Red')
      pass
    # if this point is decreasing, color it green
    elif df[dataTitle][n-1] > df[dataTitle][n] and df[dataTitle][n] > df[dataTitle][n+1]:
      ax.scatter(df[Milliseconds][n], df[dataTitle][n], color = 'Green')
      pass
    # if this graph is neither increasing nor decreasing, color it black
    else:
      ax.scatter(df[Milliseconds][n], df[dataTitle][n], color = 'Black')

In [123]:
def peaksAndValleysDF(dataTitle, df):
  """
  creates a list of all points in the given dataframe where each point is equal to the points adjacent to it
  dataTitle: string
    the name of a column in the dataframe
  df: dataframe
    the dataframe to draw from

  returns:
    a list of points, where each point is equal to the points adjacent to it
  """
  pav = []
  # for each point in the given column
  for n in range(1, len(df[dataTitle])-1):
    # if this point is increaseing, do nothing
    if df[dataTitle][n-1] < df[dataTitle][n] and df[dataTitle][n] < df[dataTitle][n+1]:
      pass
    # if this point is decreasing, do nothing
    elif df[dataTitle][n-1] > df[dataTitle][n] and df[dataTitle][n] > df[dataTitle][n+1]:
      pass
    # if this graph is neither increasing nor decreasing (it is a peak or valley), append it to pav
    else:
      pav.append((df[Milliseconds][n], df[dataTitle][n]))
  return pav

#Difference Functions

##General Difference Functions

In [124]:
def difference(df1, colTitle1, colTitle2):
  totalgraphs = 3
  y1 = df1[colTitle1]
  y2 = df1[colTitle2]
  y3 = y1-y2
  fig, ax = plt.subplots(totalgraphs, 1, figsize = (8, 6*totalgraphs))
  ax[0].plot(df1[Milliseconds],y1, label = f"{colTitle1}")
  ax[1].plot(df1[Milliseconds],y2, label = f"{colTitle2}")
  ax[2].plot(df1[Milliseconds],y3, label = f"Difference between {colTitle1} and {colTitle2}")
  for i in range (0,2):
    ax[i].legend()
  return fig

In [125]:
def differenceToFile(df1, colTitle1, colTitle2):
  fig = difference(df1, colTitle1, colTitle2)
  saveGraph(fig, f"Difference between {colTitle1} and {colTitle2}")

##Eye Difference Functions

In [126]:
def eyeDifference(dfs=[], names=[], scaledDifference = False):
  """
  outputs graphs of the Left Eye and Right Eye x- and y- eulerAngles
  dfs: list
    the dataframes for data to be drawn from
  scaledDifference: boolean
    whether or not the y bounds will be the same for both graphs

  returns: graphs of the Left Eye and Right Eye x- and y- eulerAngles
  """
  fig, ax = plt.subplots(3, 2*len(dfs), figsize = (8*2*len(dfs), 6*3))

  for q in range(len(dfs)):
    for i in range(0+2*q,2+2*q):
      if i == 0:
        colTitle1 = "LE x"
        colTitle2 = "RE x"
      if i == 1:
        colTitle1 = "LE y"
        colTitle2 = "RE y"

      y1 = dfs[q][colTitle1]
      y2 = dfs[q][colTitle2]
      y3 = y1-y2
      ax[0,i].plot(dfs[q][Milliseconds],y1, label = f"{colTitle1}")
      ax[1,i].plot(dfs[q][Milliseconds],y2, label = f"{colTitle2}")
      ax[2,i].plot(dfs[q][Milliseconds],y3, label = f"{colTitle1} - {colTitle2}")
      for j in range(0,3):
        if j == 0 or j == 1:
          ax[j,i].set_ylim(-30, 30)
        ax[j,i].legend()
        ax[j,i].set_title(names[q])

    if scaledDifference == True:
      ymin0, ymax0 = ax[2,0].get_ylim()
      ymin1, ymax1 = ax[2,1].get_ylim()
      if ymin1 < ymin0:
        ymin = ymin1
      if ymin1 > ymin0:
        ymin = ymin0
      if ymax1 < ymax0:
        ymax = ymax0
      if ymax1 > ymax0:
        ymax = ymax1
      ax[2,0].set_ylim(ymin, ymax)
      ax[2,1].set_ylim(ymin, ymax)
  return fig

In [127]:
def eyeDifferenceToFile(dfs = [], names = [], scaledDifference = False):
  fig = eyeDifference(dfs, names, scaledDifference)
  nametotal = ""
  for index in range(len(names)):
    nametotal = nametotal + names[index] + "-"
  saveGraph(fig, f"{nametotal}LE-RE Difference")

#End Function (FINAL)

In [128]:
def outputAllDataAnalysis(dfs = [], names = [], skinnyGraph = True, scaled = True, individualColPairs = False, scaledDifference = False):
  symmetryValuesToFile(dfs, names)
  averageValuesToFile(dfs, names, individualColPairs)
  graphAllToFile(dfs, names, True, False)
  eyeDifferenceToFile(dfs, names, scaledDifference)
  BlinkGraphsToFile(dfs, names)
  if len(dfs) > 1:
    compareSymmetryGraphsToFile(dfs, names, skinnyGraph, scaled)

#Running Functions

What each function does, basically

For specific information, refer to the specific function in the Table of Context

In [129]:
outputAllDataAnalysis(dfs, dfsNames, True, False, False)

In [130]:
# if you want to zoom in on a graph, you can enable or disable zooming by
# uncommenting one of the lines below and running this cell
# Enable Zoom:
# mpld3.enable_notebook()
# Disable Zoom:
# mpld3.disable_notebook()

In [ ]:
# commented functions
# "if you want to run a specific function, uncomment the line of the function you want to run,
# and specify the inputs of the function"